# Read Data

In [1]:
import pickle as pkl
input_file = open('./Gen_data/14_Network_input.pkl','r')
pro_input = pkl.load(input_file)
ehr_input = pkl.load(input_file)
Y_org=pkl.load(input_file)
input_file.close()

In [2]:
input_file = open('./Gen_data/14_embedding_matrix.pkl','r')
pro_embedding_matrix = pkl.load(input_file)
ehr_embedding_matrix = pkl.load(input_file)
input_file.close()

# 准备输入

# 切分训练集

In [3]:
from numpy.random import shuffle
import numpy as np

In [4]:
from keras.utils.np_utils import to_categorical

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [5]:
Y = to_categorical(Y_org)

In [6]:
SPLIT_point = int(0.85*len(Y_org))
seq_index = range(len(Y_org))
shuffle(seq_index)

In [7]:
x_train ,y_train =pro_input[seq_index[:SPLIT_point]], Y[seq_index[:SPLIT_point]]
x_valid ,y_valid = pro_input[seq_index[SPLIT_point:]], Y[seq_index[SPLIT_point:]]

x_train_ehr ,x_valid_ehr =ehr_input[seq_index[:SPLIT_point]], ehr_input[seq_index[SPLIT_point:]]

In [8]:
print np.shape(x_train[0]),y_train[0]

(800,) [ 1.  0.]


# Network

In [9]:
from keras.layers import Embedding,InputLayer
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten,Merge

In [10]:
from keras import backend as K
from keras import optimizers as opt

In [11]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializations
# Attention GRU network
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [12]:
# temp
import numpy as np
pro_input=np.zeros((800,4096),dtype = int )
ehr_input=np.zeros((800,4096),dtype=int)

EMBEDDING_DIM =100
MAX_SEQUENCE_LENGTH = 800  
nb_words =4096   #字典的len(keys())



In [29]:
embedding_layer = Embedding(nb_words,
  EMBEDDING_DIM,
  weights=[pro_embedding_matrix],
  input_length=MAX_SEQUENCE_LENGTH,
  trainable=True)
ehr_embedding_layer = Embedding(nb_words,
  EMBEDDING_DIM,
  weights=[pro_embedding_matrix],
  input_length=MAX_SEQUENCE_LENGTH,
  trainable=True)

left_input = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
left = embedding_layer(left_input)

right_input = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
right = ehr_embedding_layer(left_input)

l_merge = Merge( mode='concat')([left, right])
l_bid = Bidirectional(LSTM(100,return_sequences=True))(l_merge)
l_att = AttLayer()(l_bid)
l_den = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_att)

model =Model([left_input,right_input],l_den)
model.compile(loss='mse', optimizer='rmsprop',metrics=['acc'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 800)           0                                            
____________________________________________________________________________________________________
embedding_7 (Embedding)          (None, 800, 100)      409600      input_8[0][0]                    
____________________________________________________________________________________________________
embedding_8 (Embedding)          (None, 800, 100)      409600      input_8[0][0]                    
____________________________________________________________________________________________________
merge_4 (Merge)                  (None, 800, 200)      0           embedding_7[0][0]                
                                                                   embedding_8[0][0]       

# 训练模型

In [55]:
model.fit([x_train, x_train_ehr], y_train, batch_size=128, nb_epoch=1, validation_data=([x_valid, x_valid_ehr], y_valid))

Train on 3585 samples, validate on 633 samples
Epoch 1/1
3585/3585 [==============================] - 47s - loss: 0.5696 - acc: 0.5607 - val_loss: 0.6348 - val_acc: 0.5055


# 构造输出模型

In [62]:
model_getvec = Model([left_input,right_input],l_att)

In [64]:
predict  = model_getvec.predict([x_train,x_train_ehr])